# Linear Regression

Object: Predict the house price in USA

In [1]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.log4j._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.43.118:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1546201001437)
SparkSession available as 'spark'


import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.log4j._


In [2]:
Logger.getLogger("org").setLevel(Level.ERROR)
val file ="Machine_Learning_Sections/Regression/USA_Housing.csv"
var df = spark.read.option("header","true").option("inferSchema","true").format("csv").load(file)
val columnsNames = df.columns

file: String = Machine_Learning_Sections/Regression/USA_Housing.csv
df: org.apache.spark.sql.DataFrame = [AvgAreaIncome: string, AvgAreaHouseAge: string ... 5 more fields]
columnsNames: Array[String] = Array(AvgAreaIncome, AvgAreaHouseAge, AvgAreaNumberOfRooms, AvgAreaNumberOfBedrooms, AreaPopulation, Price, Address)


In [3]:
columnsNames

res0: Array[String] = Array(AvgAreaIncome, AvgAreaHouseAge, AvgAreaNumberOfRooms, AvgAreaNumberOfBedrooms, AreaPopulation, Price, Address)


In [4]:
df.printSchema

root
 |-- AvgAreaIncome: string (nullable = true)
 |-- AvgAreaHouseAge: string (nullable = true)
 |-- AvgAreaNumberOfRooms: double (nullable = true)
 |-- AvgAreaNumberOfBedrooms: double (nullable = true)
 |-- AreaPopulation: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Address: string (nullable = true)



In [5]:
// chang de type
df = df.selectExpr("cast("+columnsNames(0).toString+ " as double)",
"cast("+columnsNames(1).toString+ " as double)",
"cast("+columnsNames(2).toString+ " as double)",
"cast("+columnsNames(3).toString+ " as double)",
"cast("+columnsNames(4).toString+ " as double)",
"cast("+columnsNames(5).toString+ " as double)",
"cast("+columnsNames(6).toString+ " as string)")

df: org.apache.spark.sql.DataFrame = [AvgAreaIncome: double, AvgAreaHouseAge: double ... 5 more fields]


In [6]:
df.printSchema

root
 |-- AvgAreaIncome: double (nullable = true)
 |-- AvgAreaHouseAge: double (nullable = true)
 |-- AvgAreaNumberOfRooms: double (nullable = true)
 |-- AvgAreaNumberOfBedrooms: double (nullable = true)
 |-- AreaPopulation: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Address: string (nullable = true)



In [7]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector

val df1 = df.select(df("Price").as("label"),
                    $"AvgAreaIncome",
                    $"AvgAreaHouseAge",
                    $"AvgAreaNumberofRooms",
                    $"AvgAreaNumberofBedrooms",
                    $"AreaPopulation",
                    $"Address")

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
df1: org.apache.spark.sql.DataFrame = [label: double, AvgAreaIncome: double ... 5 more fields]


In [8]:
df1.printSchema

root
 |-- label: double (nullable = true)
 |-- AvgAreaIncome: double (nullable = true)
 |-- AvgAreaHouseAge: double (nullable = true)
 |-- AvgAreaNumberofRooms: double (nullable = true)
 |-- AvgAreaNumberofBedrooms: double (nullable = true)
 |-- AreaPopulation: double (nullable = true)
 |-- Address: string (nullable = true)



In [22]:
df1.count

res12: Long = 10000


In [30]:
df1.filter(row => row.anyNull).count

res17: Long = 5000


In [31]:
df = df1.na.drop()

df: org.apache.spark.sql.DataFrame = [label: double, AvgAreaIncome: double ... 5 more fields]


In [32]:
val assembler = new VectorAssembler().setInputCols(Array("AvgAreaIncome",
		 "AvgAreaHouseAge",
		 "AvgAreaNumberofRooms",
		 "AvgAreaNumberofBedrooms",
		 "AreaPopulation")).setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_8f2547fcc665


In [33]:
val output = assembler.transform(df).select($"label",$"features")

output: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [34]:
val lr = new LinearRegression()

lr: org.apache.spark.ml.regression.LinearRegression = linReg_cc5ea58ae11f


In [35]:
val lrModel = lr.fit(output)

2018-12-30 21:42:50 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2018-12-30 21:42:50 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
2018-12-30 21:42:51 WARN  LAPACK:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
2018-12-30 21:42:51 WARN  LAPACK:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_cc5ea58ae11f


In [36]:
// Print the coefficients
println(s"Coefficients: ${lrModel.coefficients}")

Coefficients: [21.578049448352026,165637.02694091276,120659.94881629614,1651.1390539904344,15.200743923741493]


In [37]:
//intercept for linear regression
println(s"Intercept: ${lrModel.intercept}")

Intercept: -2637299.033328577


In [39]:
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@3ec5d0e1


In [40]:
trainingSummary.predictions.show(8)

+------------------+--------------------+------------------+
|             label|            features|        prediction|
+------------------+--------------------+------------------+
|1059033.5578701235|[79545.4585743167...|1223847.0427535456|
|  1505890.91484695|[79248.6424548256...|1494937.6916173412|
|1058987.9878760849|[61287.0671786567...|1253016.7460814407|
|1260616.8066294468|[63345.2400462279...|1121224.0676507396|
| 630943.4893385402|[59982.1972257080...| 845388.7662952547|
|1068138.0743935304|[80175.7541594853...| 1068839.155724872|
|1502055.8173744078|[64698.4634278877...| 1670159.612035254|
|1573936.5644777215|[78394.3392775308...|1569962.3467741245|
+------------------+--------------------+------------------+
only showing top 8 rows



In [42]:
// Coeff deter
trainingSummary.r2

res23: Double = 0.9180238195089548
